##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# What's new in Gemini-1.5-pro-002 and Gemini-1.5-flash-002

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/New_in_002.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook explores the new options added with the 002 versions of the 1.5 series models:

* Candidate count
* Presence and frequency penalties
* Response logprobs

## Setup

Install a `002` compatible version of the SDK:

In [1]:
%pip install -q "google-genai>=1.7.0"

import the package and give it your API-key

In [2]:
from google import genai

In [3]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

Import other packages.

In [4]:
from IPython.display import display, Markdown, HTML

Check available 002 models

In [6]:
for model in genai.list_models():
  if '002' in model.name:
    print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002
models/imagen-3.0-generate-002


In [7]:
model_name = "models/gemini-1.5-flash-002"
test_prompt="Why don't people have tails"

## Quick refresher on `config` [Optional]

In [8]:
from google.genai import types

response = client.models.generate_content(
    model=model_name,
    contents="hello",
    config=types.GenerateContentConfig(
        max_output_tokens=5,
        temperature=1.0
    )
)

Note:

* Each `generate_content` request is sent with a `config` (`chat.send_message` uses `generate_content`).
* You can set the `config` by passing it in the arguments to `generate_content` (or `chat.send_message`).
* Any `config` attributes set in `generate_content` override the attributes set on the model.
* You can pass the `generation_config` as either a Python `dict`, or a `genai.GenerationConfig`.
* If you're ever unsure about the parameters of `generation_config` check `genai.GenerationConfig`.

## Candidate count

With 002 models you can now use `candidate_count > 1`.

In [12]:
response = client.models.generate_content(
    model=model_name,
    contents=test_prompt,
    config=types.GenerateContentConfig(
        candidate_count=2
    )
)

But note that the `.text` quick-accessor only works for the simple 1-candidate case.

In [13]:
try:
  response.text # Fails with multiple candidates, sorry!
except ValueError as e:
  print(e)

With multiple candidates you have to handle the list of candidates yourself:

In [14]:
for candidate in response.candidates:
  display(Markdown(candidate.content.parts[0].text))
  display(Markdown("-------------"))


Humans don't have tails because of evolutionary changes over millions of years.  Our primate ancestors had tails, but as humans evolved, the genes responsible for tail development were suppressed.  This wasn't a single event, but a gradual process driven by natural selection.  While the exact reasons are still being researched, several hypotheses contribute:

* **Bipedalism:**  As our ancestors transitioned to walking upright, a tail became less advantageous.  Tails are helpful for balance in quadrupedal animals, but for bipeds, they would have been more of a hindrance.  The energy required to maintain and control a tail might have been better allocated to other aspects of survival and reproduction.

* **Reduced need for arboreal locomotion:**  Our ancestors spent less time in trees and more time on the ground.  Tails are useful for gripping branches, so as arboreal life decreased, the selective pressure for a tail lessened.

* **Genetic mutations:**  Random genetic mutations that affected tail development may have been favored by natural selection, especially if they offered other benefits or didn't create significant disadvantages.  These mutations could have gradually reduced tail size until it was essentially vestigial, and then eventually disappeared altogether.

It's important to note that while humans typically lack external tails, the coccyx (tailbone) is a remnant of our tailed ancestry.  It's a vestigial structure, meaning it's a leftover from our evolutionary past that no longer serves its original function.  In rare cases, humans are born with rudimentary tails, which usually require surgical removal.  This shows that the genetic mechanisms for tail development haven't completely disappeared, but they are usually suppressed.


-------------

Humans don't have tails because of evolutionary changes over millions of years.  Our ancestors had tails, but as humans evolved, the genes controlling tail development were suppressed.  The exact reasons are complex and not fully understood, but several theories exist:

* **Loss of Functionality:**  As our primate ancestors transitioned to upright walking and arboreal life became less crucial, the need for a prehensile tail diminished.  Natural selection favored individuals with less prominent or even absent tails, as they may have offered no significant survival advantage and might even have presented disadvantages (like increased vulnerability during falls).

* **Developmental Changes:**  Changes in the timing and expression of genes during embryonic development led to the shortening of the tail.  The coccyx (tailbone) is a vestigial remnant of this tail.

* **Energetic Costs:** Maintaining a tail requires energy.  As humans evolved larger brains and more complex behaviors, resources may have been better allocated to other functions than maintaining a tail.

In short, the absence of a tail in humans is a result of a gradual evolutionary process where the selective pressures favored individuals with reduced or absent tails, leading to the eventual suppression of tail development in our species.  It's a testament to the power of natural selection and adaptation over vast timescales.


-------------

The response contains multiple full `Candidate` objects.

In [17]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="Humans don't have tails because of evolutionary changes over millions of years.  Our primate ancestors had tails, but as humans evolved, the genes responsible for tail development were suppressed.  This wasn't a single event, but a gradual process driven by natural selection.  While the exact reasons are still being researched, several hypotheses contribute:\n\n* **Bipedalism:**  As our ancestors transitioned to walking upright, a tail became less advantageous.  Tails are helpful for balance in quadrupedal animals, but for bipeds, they would have been more of a hindrance.  The energy required to maintain and control a tail might have been better allocated to other aspects of survival and reproduction.\n\n* **Reduced need for arboreal locomotion:**

## Penalties

The `002` models expose `penalty` arguments that let you affect the statistics of output tokens.

### Presence penalty

The `presence_penalty` penalizes tokens that have already been used in the output, so it induces variety in the model's output. This is detectible if you count the unique words in the output.

Here's a function to run a prompt a few times and report the fraction of unique words (words don't map perfectly to tokens but it's a simple way to see the effect).

In [18]:
from statistics import mean

In [19]:
def unique_words(prompt, N=10):
  responses = []
  vocab_fractions = []

  for n in range(N):
    response = client.models.generate_content(
        model=model_name,
        contents=prompt
    )

    responses.append(response)

    words = response.text.lower().split()
    score = len(set(words))/len(words)
    print(score)
    vocab_fractions.append(score)

  return vocab_fractions

In [20]:
prompt='Tell me a story'

In [21]:
# baseline
v = unique_words(prompt)

0.562962962962963
0.5968586387434555
0.5833333333333334
0.5846867749419954
0.5643153526970954
0.5961995249406176
0.6004016064257028
0.6240786240786241
0.6356382978723404
0.6209476309226932


In [22]:
mean(v)

0.5969422746918821

In [ ]:
# the penalty encourages diversity in the oputput tokens.
v = unique_words(prompt, generation_config=dict(presence_penalty=1.999))

0.6214833759590793
0.5617529880478087
0.5894495412844036
0.5789473684210527
0.5781990521327014
0.6389684813753582
0.6061320754716981
0.5727482678983834
0.5864485981308412
0.565410199556541


In [ ]:
mean(v)

0.5899539948277868

In [ ]:
# a negative penalty discourages diversity in the output tokens.
v = unique_words(prompt, generation_config=dict(presence_penalty=-1.999))

0.5555555555555556
0.6472148541114059
0.5839598997493735
0.6132075471698113
0.5858369098712446
0.5823389021479713
0.5895691609977324
0.5978021978021978
0.5604166666666667
0.5741626794258373


In [ ]:
mean(v)

0.5890064373497796

The `presence_penalty` has a small effect on the vocabulary statistics.

### Frequency Penalty

Frequency penalty is similar to the `presence_penalty` but  the penalty is multiplied by the number of times a token is used. This effect is much stronger than the `presence_penalty`.

The easiest way to see that it works is to ask the model to do something repetitive. The model has to get creative while trying to complete the task.

In [ ]:
model = genai.GenerativeModel(model_name)
response = model.generate_content(contents='please repeat "Cat" 50 times, 10 per line',
                                  generation_config=dict(frequency_penalty=1.999))

In [ ]:
print(response.text)

Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat Cat CaT CaT
Cat cat cat cat cat cat cat cat cat cat
Cat Cat Cat Cat Cat Cat Cat Cat Cat Cat
Cat Cat Cat Cat Cat Cat Cat CaT CaT CaT
Cat cat cat cat cat cat cat cat cat cat
Cat Cat Cat Cat Cat Cat Cat CaT CaT CaT
Cat CAT CAT CAT CAT CAT cAT cAT cAT CA
t Cat Cat Cat Cat cat cat cat cat CAT



Since the frequency penalty accumulates with usage, it can have a much stronger effect on the output compared to the presence penalty.

> Caution: Be careful with negative frequency penalties: A negative penalty makes a token more likely the more it's used. This positive feedback quickly leads the model to just repeat a common token until it hits the `max_output_tokens` limit (once it starts the model can't produce the `<STOP>` token).

In [ ]:
response = model.generate_content(
    prompt,
    generation_config=genai.GenerationConfig(
        max_output_tokens=400,
        frequency_penalty=-2.0))

In [ ]:
Markdown(response.text)  # the, the, the, ...

Elara, a wisp of a girl with eyes the colour of a stormy sea, lived in a lighthouse perched precariously on the edge of the Whispering Cliffs.  Her only companions were the relentless rhythm of the waves and the lonely cries of the gulls.  Her father, the lighthouse keeper, was a man of the sea, his face etched with the map of the ocean's moods.  He’d taught her the language of the waves, the the the the way the wind whispered secrets to the rocks, and the constellations that guided lost ships home.

One day, a storm unlike any Elara had ever seen descended.  The lighthouse shuddered, the wind howled like a banshee, and the waves crashed against the cliffs with the fury of a thousand angry giants.  During the tempest, a ship, its masts splintered and its sails ripped, was tossed onto the rocks below.  Elara’s father, his face grim, prepared his small, sturdy boat, defying the monstrous waves to reach the stricken vessel.

He never returned.

Days bled into weeks.  Elara, her heart a frozen wasteland, kept the light burning, a tiny, defiant flame against the overwhelming darkness.  She scanned the horizon every day, hoping, praying, for a sign, a glimpse of a familiar sail, a flicker of a known light.

One evening, a faint, almost imperceptible glow appeared on the horizon.  It was weak, flickering, but undeniably there.  It was a signal, a desperate plea for help.  Elara, her heart pounding, launched her father’s boat, her small form a mere speck against the immensity of the ocean.

The storm, though, had subsided.  The sea was calm. The glow was guiding.

She reached the ship, a small fishing trawler, battered, but afloat.  A lone figure, an old woman with silver hair, lay clinging to the

In [ ]:
response.candidates[0].finish_reason

<FinishReason.MAX_TOKENS: 2>